In [1]:
#install library

!pip install textsearch
!pip install contractions
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lenov\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lenov\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import pathlib #untuk memverifikasi keberadaan file apa pun dalam sistem
pathlib.Path().resolve()

WindowsPath('C:/Users/lenov/anaconda3/AAA_UAS')

In [3]:
#load dan view data
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

# Baca dataset tempat wisata
df = pd.read_csv("tourism_with_id.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 437 entries, 0 to 436
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Place_Id      437 non-null    int64  
 1   Place_Name    437 non-null    object 
 2   Description   437 non-null    object 
 3   Category      437 non-null    object 
 4   City          437 non-null    object 
 5   Price         437 non-null    int64  
 6   Rating        437 non-null    float64
 7   Time_Minutes  205 non-null    float64
 8   Coordinate    437 non-null    object 
 9   Lat           437 non-null    float64
 10  Long          437 non-null    float64
 11  Unnamed: 11   0 non-null      float64
 12  Unnamed: 12   437 non-null    int64  
dtypes: float64(5), int64(3), object(5)
memory usage: 44.5+ KB


In [5]:
df.head()

,Place_Id,Place_Name,Description,Category,City,Price,Rating,Time_Minutes,Coordinate,Lat,Long,Unnamed: 11,Unnamed: 12
0,1,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,Budaya,Jakarta,20000,4.6,15.0,"{'lat': -6.1753924, 'lng': 106.8271528}",-6.175392,106.827153,NaN,1
1,2,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Budaya,Jakarta,0,4.6,90.0,"{'lat': -6.137644799999999, 'lng': 106.8171245}",-6.137645,106.817125,NaN,2
2,3,Dunia Fantasi,Dunia Fantasi atau disebut juga Dufan adalah t...,Taman Hiburan,Jakarta,270000,4.6,360.0,"{'lat': -6.125312399999999, 'lng': 106.8335377}",-6.125312,106.833538,NaN,3
3,4,Taman Mini Indonesia Indah (TMII),Taman Mini Indonesia Indah merupakan suatu kaw...,Taman Hiburan,Jakarta,10000,4.5,NaN,"{'lat': -6.302445899999999, 'lng': 106.8951559}",-6.302446,106.895156,NaN,4
4,5,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...,Taman Hiburan,Jakarta,94000,4.5,60.0,"{'lat': -6.12419, 'lng': 106.839134}",-6.124190,106.839134,NaN,5


In [6]:
# Buat fungsi untuk mengubah dataset menjadi matrix
def get_title_from_index(index):
    return df[df.index == index]["Place_Name"].values[0]

In [7]:
def get_index_from_title(title):
    return df[df.Place_Name == title]["Place_Id"].values[0]

In [8]:
df["combined_features"] = df["Description"] + " " + df["Category"] + " " + df["City"]

In [9]:
df.iloc[0].combined_features

'Monumen Nasional atau yang populer disingkat dengan Monas atau Tugu Monas adalah monumen peringatan setinggi 132 meter (433 kaki) yang didirikan untuk mengenang perlawanan dan perjuangan rakyat Indonesia untuk merebut kemerdekaan dari pemerintahan kolonial Hindia Belanda. Pembangunan monumen ini dimulai pada tanggal 17 Agustus 1961 di bawah perintah presiden Soekarno dan dibuka untuk umum pada tanggal 12 Juli 1975. Tugu ini dimahkotai lidah api yang dilapisi lembaran emas yang melambangkan semangat perjuangan yang menyala-nyala. Monumen Nasional terletak tepat di tengah Lapangan Medan Merdeka, Jakarta Pusat. Budaya Jakarta'

In [10]:
#BUAT SISTEM REKOMENDASI

import nltk
import re
import numpy as np
import contractions

stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    doc = contractions.fix(doc)
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    #filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(list(df['combined_features']))
len(norm_corpus)

437

In [11]:
#EKSTRAK TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(ngram_range=(1, 2), min_df=2)
tfidf_matrix = tf.fit_transform(norm_corpus)
tfidf_matrix.shape

(437, 6883)

In [12]:
# Hitung Kemiripan Dokumen

from sklearn.metrics.pairwise import cosine_similarity

doc_sim = cosine_similarity(tfidf_matrix)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,...,427,428,429,430,431,432,433,434,435,436
0,1.000000,0.056345,0.121332,0.043982,0.028871,0.026768,0.045482,0.016387,0.051150,0.026915,...,0.187716,0.051643,0.022321,0.030459,0.037396,0.065244,0.057110,0.049843,0.037196,0.009002
1,0.056345,1.000000,0.049264,0.050099,0.042804,0.064113,0.035485,0.025187,0.129625,0.039447,...,0.030992,0.059314,0.044906,0.028648,0.020757,0.077165,0.033392,0.026778,0.014085,0.081544
2,0.121332,0.049264,1.000000,0.165668,0.150856,0.464421,0.059870,0.083126,0.251460,0.051984,...,0.023687,0.037173,0.046920,0.077743,0.074406,0.051815,0.075939,0.054578,0.050110,0.013327
3,0.043982,0.050099,0.165668,1.000000,0.068982,0.166373,0.059479,0.035295,0.075736,0.053044,...,0.026485,0.017053,0.078703,0.077571,0.132967,0.028804,0.134761,0.042435,0.059828,0.030583
4,0.028871,0.042804,0.150856,0.068982,1.000000,0.143611,0.020433,0.082521,0.164943,0.027008,...,0.047871,0.013342,0.190574,0.044862,0.033359,0.022632,0.051623,0.084718,0.023627,0.030017


In [13]:
#MENDAPATKAN JUDUL

PlaceName_list = df['Place_Name'].values
PlaceName_list, PlaceName_list.shape

(array(['Monumen Nasional', 'Kota Tua', 'Dunia Fantasi',
        'Taman Mini Indonesia Indah (TMII)', 'Atlantis Water Adventure',
        'Taman Impian Jaya Ancol', 'Kebun Binatang Ragunan',
        'Ocean Ecopark', 'Pelabuhan Marina', 'Pulau Tidung',
        'Pulau Bidadari', 'Pulau Pari', 'Pulau Pramuka', 'Pulau Pelangi',
        'Pasar Seni', 'Jembatan Kota Intan', 'Museum Fatahillah',
        'Museum Bank Indonesia', 'Kidzania', 'Museum Taman Prasasti',
        'Museum Wayang', 'Masjid Istiqlal', 'Gereja Katedral',
        'Museum Nasional', 'Pasar Tanah Abang', 'Pecinan Glodok',
        'Sea World', 'Wisata Agro Edukatif Istana Susu Cibugary',
        'Wisata Kuliner Pecenongan', 'Taman Menteng',
        'Wisata Alam Mangrove Angke', 'Setu Babakan', 'Taman Suropati',
        'Pasar Taman Puring', 'Grand Indonesia Mall',
        'Skyrink - Mall Taman Anggrek', 'Bumi Perkemahan Cibubur',
        'Istana Negara Republik Indonesia',
        'Museum Macan (Modern and Contemporary Art i

In [14]:
#MENDAPATKANN INDEX

PlaceName_id = np.where(PlaceName_list == 'Kota Tua')[0][0]
PlaceName_id

1

In [15]:
#AMBIL KESAMAAN / SIMILARITIES

PlaceName_similarities = doc_sim_df.iloc[PlaceName_id].values
PlaceName_similarities

array([0.05634541, 1.        , 0.04926384, 0.05009942, 0.04280443,
       0.06411301, 0.03548464, 0.02518735, 0.12962483, 0.0394469 ,
       0.05028222, 0.04710723, 0.01388418, 0.04296712, 0.03286994,
       0.13635234, 0.19066043, 0.05448176, 0.02979126, 0.12520333,
       0.16386927, 0.04209643, 0.05794018, 0.14285456, 0.03236682,
       0.0763524 , 0.04047749, 0.01648488, 0.11605893, 0.03986114,
       0.06884228, 0.08034713, 0.04699582, 0.02291766, 0.03615642,
       0.0482476 , 0.04119714, 0.0325949 , 0.09896613, 0.04726311,
       0.19379685, 0.18347112, 0.05144832, 0.04451278, 0.04823821,
       0.037208  , 0.06700493, 0.10757674, 0.04942559, 0.1098796 ,
       0.08140345, 0.06336595, 0.08373473, 0.05343291, 0.03678187,
       0.04343433, 0.05984558, 0.06930272, 0.09328303, 0.1065232 ,
       0.0598454 , 0.03443885, 0.10400559, 0.0842626 , 0.13339096,
       0.10831954, 0.03888485, 0.06187074, 0.04199729, 0.03416654,
       0.01925345, 0.03617535, 0.09722301, 0.11148526, 0.03699

In [16]:
#AMBIL INDEX PARIWISATA TOP 5 

similar_PlaceName_id = np.argsort(-PlaceName_similarities)[1:6]
similar_PlaceName_id

array([ 40,  16,  41, 117,  20], dtype=int64)

In [17]:
# Buat fungsi rekomendasi tempat wisata untuk merekomendasikan 5 tempat wisata serupa teratas untuk pariwisata apa pun

def rekomendasi_tempat_wisata(PlaceName_title, PlaceName=PlaceName_list, doc_sims=doc_sim_df):
    # find PlaceName id
    PLaceName_id = np.where(PlaceName ==PlaceName_title)[0][0]
    # get PlaceName similarities
    PlaceName_similarities = doc_sims.iloc[PlaceName_id].values
    # get top 5 similar PlaceName IDs
    similar_PlaceName_id = np.argsort(-PlaceName_similarities)[1:6]
    # get top 5 PlaceName
    similar_PlaceName = PlaceName[similar_PlaceName_id]
    # return the top 5 PlaceName
    return similar_PlaceName

In [18]:
# MASUKKAN LIST NAMA TEMPAT WISATA

popular_tourism = ['Monumen Nasional', 'Kota Tua', 'Dunia Fantasi',
        'Taman Mini Indonesia Indah (TMII)', 'Atlantis Water Adventure',
        'Taman Impian Jaya Ancol', 'Kebun Binatang Ragunan',
        'Ocean Ecopark', 'Pelabuhan Marina', 'Pulau Tidung',
        'Pulau Bidadari', 'Pulau Pari', 'Pulau Pramuka', 'Pulau Pelangi',
        'Pasar Seni', 'Jembatan Kota Intan', 'Museum Fatahillah',
        'Museum Bank Indonesia', 'Kidzania', 'Museum Taman Prasasti',
        'Museum Wayang', 'Masjid Istiqlal', 'Gereja Katedral',
        'Museum Nasional', 'Pasar Tanah Abang', 'Pecinan Glodok',
        'Sea World', 'Wisata Agro Edukatif Istana Susu Cibugary',
        'Wisata Kuliner Pecenongan', 'Taman Menteng',
        'Wisata Alam Mangrove Angke', 'Setu Babakan', 'Taman Suropati',
        'Pasar Taman Puring', 'Grand Indonesia Mall',
        'Skyrink - Mall Taman Anggrek', 'Bumi Perkemahan Cibubur',
        'Istana Negara Republik Indonesia',
        'Museum Macan (Modern and Contemporary Art in Nusantara)',
        'Galeri Nasional Indonesia', 'Museum Bahari Jakarta',
        'Museum Seni Rupa dan Kramik', 'The Escape Hunt',
        'Monumen Selamat Datang', 'Jakarta Aquarium dan Safari',
        'Pulau Semak Daun', 'Taman Situ Lembang', 'Taman Ismail Marzuki',
        'Galeri Indonesia Kaya', 'Museum Joang 45', 'Jakarta Planetarium',
        'Kampung Cina', 'Rumah Sipitung', 'Taman Agrowisata Cilangkap',
        'Klenteng Jin De Yuan', 'Pantai Ancol', 'Taman Lapangan Banteng',
        'Taman Ayodya', 'Museum Sumpah Pemuda', 'Museum Tekstil',
        'Tugu Proklamasi', 'Taman Legenda Keong Emas',
        'Museum Kebangkitan Nasional', 'Museum Sasmita Loka Ahmad Yani',
        'Museum Basoeki Abdullah', 'Museum Layang-layang',
        'Margasatwa Muara Angke', 'Pasar Petak Sembilan',
        'Freedom Library', 'Perpustakaan Nasional',
        'Cibubur Garden Diary (Cibugary)',
        'Waterboom PIK (Pantai Indah Kapuk)', 'Museum Perangko',
        'Museum Tengah Kebun', 'SnowBay Waterpark', 'Taman Cattleya',
        'Taman Hutan Tebet', 'Hutan Kota Srengseng', 'Taman Spathodea',
        'Plaza Indonesia', 'Mall Thamrin City', 'Museum Satria Mandala',
        'Alive Museum Ancol', 'Kawasan Kuliner BSM',
        'Taman Pintar Yogyakarta', 'Keraton Yogyakarta',
        'Sindu Kusuma Edupark (SKE)',
        'Museum Benteng Vredeburg Yogyakarta', 'De Mata Museum Jogja',
        'Kampung Wisata Taman Sari', 'Situs Warungboto',
        'Nol Kilometer Jl.Malioboro', 'Gembira Loka Zoo', 'Sumur Gumuling',
        'Desa Wisata Sungai Code Jogja Kota',
        'Alun Alun Selatan Yogyakarta', 'Monumen Yogya Kembali',
        'Taman Pelangi Yogyakarta', 'Kampung Wisata Kadipaten',
        'Taman Budaya Yogyakarta', 'Kampung Wisata Sosro Menduran',
        'Monumen Batik Yogyakarta', 'Tugu Pal Putih Jogja',
        'Tebing Breksi', 'Candi Donotirto', 'Kawasan Malioboro',
        'Bangsal Pagelaran', 'Embung Tambakboyo', 'Hutan Pinus Pengger',
        'Monumen Serangan Umum 1 Maret', 'Puncak Pinus Becici',
        'Bukit Bintang Yogyakarta', 'Gedung Agung Yogyakarta',
        'The Lost World Castle', 'Monumen Sanapati',
        'Jurang Tembelan Kanigoro',
        'The World Landmarks - Merapi Park Yogyakarta',
        'Museum Sonobudoyo Unit I', 'Lava Bantal', 'Bukit Panguk Kediwung',
        'Candi Prambanan', 'Watu Goyang', 'Kampung Wisata Rejowinangun',
        'Kauman Pakualaman Yogyakarta',
        'Alun-alun Utara Keraton Yogyakarta', 'Pantai Goa Cemara',
        'Blue Lagoon Jogja', 'Gumuk Pasir Parangkusumo',
        'Bukit Lintang Sewu', 'Studio Alam Gamplong', 'Watu Lumbung',
        'Air Terjun Kedung Pedut', 'Puncak Kebun Buah Mangunan',
        'Desa Wisata Gamplong', 'Kawasan Wisata Sosrowijayan',
        'Grojogan Watu Purbo Bangunrejo', 'Wisata Alam Kalibiru',
        'Jogja Exotarium', 'Puncak Gunung Api Purba - Nglanggeran',
        'Bendung Lepen', 'Bunker Kaliadem Merapi', 'Ledok Sambi',
        'Seribu Batu Songgo Langit', 'Goa Jomblang',
        'Desa Wisata Rumah Domes/Teletubbies', 'Bukit Wisata Pulepayung',
        'Bentara Budaya Yogyakarta (BBY)', 'Goa Rancang Kencono',
        'Goa Cerme', 'Wisata Kraton Jogja', 'Desa Wisata Kelor',
        'Pantai Patihan', 'Pantai Kukup', 'Pantai Ngrawe (Mesra)',
        'Pantai Ngobaran', 'Pantai Pulang Sawal', 'Pantai Baron',
        'Pantai Pok Tunggal', 'Hutan Pinus Asri',
        'Pasar Kebon Empring Bintaran', 'Bukit Paralayang, Watugupit',
        'Kampung Wisata Dipowinatan', 'Watu Mabur Mangunan',
        'Pintoe Langit Dahromo', 'Green Village Gedangsari',
        'Kebun Teh Nglinggo', 'Geoforest Watu Payung Turunan',
        'Pantai Timang', 'Puncak Segoro', 'Candi Borobudur', 'Candi Ijo',
        'Pantai Indrayanti', 'Pantai Nglambor', 'Pasar Beringharjo',
        'Museum Ullen Sentalu', 'Museum Gunung Merapi',
        'Pantai Parangtritis', 'Goa Pindul', 'Candi Ratu Boko',
        'Pantai Depok Jogja', 'Pantai Sundak',
        'Hutan Mangrove Kulon Progo',
        'Jogja Bay Pirates Adventure Waterpark', 'Pantai Sepanjang',
        'Pantai Krakal', 'Pantai Glagah', 'Pantai Siung',
        'Pantai Sadranan', 'Pantai Nguluran', 'Pantai Samas',
        'Pantai Drini', 'Air Terjun Sri Gethuk', 'Candi Sewu',
        'Pantai Wediombo', 'Pantai Jogan', 'Pantai Ngrenehan',
        'Pantai Jungwok', 'Pantai Greweng', 'Pantai Sedahan',
        'Pantai Watu Kodok', 'Pantai Kesirat', 'Pantai Ngandong',
        'Galaxy Waterpark Jogja', 'Desa Wisata Pulesari',
        'Desa Wisata Tembi', 'Wisata Kaliurang', 'Heha Sky View',
        'Taman Sungai Mudal', 'Pantai Sanglen', 'Pantai Congot',
        'GunungTangkuban perahu', 'Jalan Braga', 'Gedung Sate',
        'Trans Studio Bandung', 'Taman Hutan Raya Ir. H. Djuanda',
        'Farm House Susu Lembang', 'Kebun Binatang Bandung', 'Kawah Putih',
        'Tebing Karaton', 'Dusun Bambu', 'Museum Geologi Bandung',
        'Museum Konferensi Asia Afrika', 'Masjid Raya Bandung',
        'Dago Dreampark', 'Orchid Forest Cikole', 'Kawasan Punclut',
        'Alun-Alun Kota Bandung', 'Museum Sri Baduga', 'Rabbit Town',
        'Stone Garden Citatah', 'The Great Asia Africa', 'Bukit Moko',
        'Gunung Papandayan', 'Amazing Art World', 'Gua Belanda',
        'Taman Balai Kota Bandung', 'Panghegar Waterboom Bandung',
        'Gunung Manglayang', 'Lembang Park & Zoo',
        'Taman Lalu Lintas Ade Irma Suryani Nasution',
        'Lawangwangi Creative Space', 'Curug Dago',
        'Kampung Korea Bandung', 'Kampung Batu Malakasari', 'Jendela Alam',
        'Curug Tilu Leuwi Opat', 'Kiara Artha Park',
        'NuArt Sculpture Park', 'Upside Down World Bandung',
        'Pesona Nirwana Waterpark & Cottages', 'Taman Lansia',
        'Kampoeng Tulip', 'Selasar Sunaryo Art Space',
        'Teras Cikapundung BBWS', 'Museum Barli', 'Wisata Batu Kuda',
        'Monumen Bandung Lautan Api', 'Museum Gedung Sate',
        'Monumen Perjuangan Rakyat Jawa Barat', 'Museum Pos Indonesia',
        'Ciwangun Indah Camp Official', 'Wot Batu', 'Curug Batu Templek',
        'Gereja Katedral Santo Petrus Bandung',
        'Museum Mandala Wangsit Siliwangi', 'Caringin Tilu',
        'Panama Park 825', 'Sudut Pandang Bandung',
        'Bumi Perkemahan Batu Kuda', 'Bukit Bintang', 'Taman Vanda',
        'Taman Budaya Jawa Barat', 'Curug Bugbrug',
        'Babakan Siliwangi City Forest Path Bandung', 'Curug Cipanas',
        'Taman Jomblo', 'Peta Park', 'Masjid Daarut Tauhiid Bandung',
        'Masjid Agung Trans Studio Bandung',
        'Gereja Tiberias Indonesia Bandung', 'Tektona Waterpark',
        'Curug Cilengkrang', 'Jembatan Pasupati', 'Pasar Baru',
        'Taman Sejarah Bandung', 'Roemah Seni Sarasvati',
        'Puspa Iptek Sundial', 'Masjid Salman ITB', 'Curug Aseupan',
        'Museum Pendidikan Nasional', 'Water Park Bandung Indah',
        'Curug Anom', 'Taman Badak', 'Taman Film', 'Museum Nike Ardilla',
        'Batununggal Indah Club', 'Mountain View Golf Club',
        'Gunung Lalakon', 'Sendang Geulis Kahuripan', 'Sanghyang Heuleut',
        'Lereng Anteng Panoramic Coffee Place', 'Masjid Pusdai',
        'Masjid Al-Imtizaj', 'Situ Cileunca', 'Sunrise Point Cukul',
        'Sungai Palayangan', 'Perkebunan Teh Malabar',
        'Pemandian Air Panas Ciater', 'Observatorium Bosscha',
        'Taman Bunga Cihideung', 'The Lodge Maribaya',
        'Taman Hutan Raya Ir. H. Juanda', 'Taman Begonia', 'Tafso Barn',
        'Curug Luhur Waterfall', 'Curug Malela', 'Curug Cimahi',
        'Situ Patenggang', 'Kawah Rengganis Cibuni',
        'Pemandian Air Panas Cimanggu', 'Glamping Lakeside Rancabali',
        'Bukit Jamur', 'Kebun Tanaman Obat Sari Alam', 'Happyfarm Ciwidey',
        'Saung Angklung Mang Udjo', 'Taman Kupu-Kupu Cihanjuang',
        'Taman Miniatur Kereta Api', 'Gua Pawon', 'Bukit Gantole Cililin',
        'Bandros City Tour', 'Kyotoku Floating Market', 'Rainbow Garden',
        'Kota Mini', 'Chingu Cafe Little Seoul', 'Candi Gedong Songo',
        'Grand Maerakaca', 'Kampung Pelangi', 'Lawang Sewu',
        'Sam Poo Kong Temple', 'Desa Wisata Lembah Kalipancur',
        'Hutan Wisata Tinjomoyo Semarang', 'Taman Kasmaran',
        'Pantai Baruna', 'Pantai Marina', 'Kampoeng Djadhoel Semarang',
        'Indonesia Kaya Park', 'Taman Pandanaran', 'Pura Giri Natha',
        'Wisata Mangrove Tapak', 'Pantai Cipta',
        'Old City 3D Trick Art Museum',
        'GPIB Immanuel Semarang (Gereja Blenduk)', 'Taman Srigunting',
        'Semarang Contemporary Art Gallery', 'Umbul Sidomukti',
        'Saloka Theme Park', 'Wisata Alam Wana Wisata Penggaron',
        'Masjid Kapal Semarang', 'Taman Bunga Celosia',
        'Air Terjun Kali Pancur', 'Wisata Kampung Krisan Clapar',
        'Danau Rawa Pening', 'Pantai Maron',
        'Water Blaster Bukit Candi Golf', 'Waduk Jatibarang',
        'Tirto Argo Siwarak', 'Wisata Lereng Kelir', 'Kampoeng Rawa',
        'Monumen Palagan Ambarawa', 'Benteng Pendem',
        'Wisata Eling Bening', 'Gua Maria Kerep Ambarawa',
        'Museum Kereta Ambarawa', 'Hutan Pinus Kayon',
        'Kota Lama Semarang', 'Semarang Chinatown', 'La Kana Chapel',
        'Jembatan Biru Ambarawa', 'Goa Rong', 'Masjid Agung Ungaran',
        'Tugu Muda Semarang', 'Kampoeng Kopi Banaran', 'Taman Tabanas',
        'Kampung Tematik Jawi', 'Kampung Batik Gedong Semarang',
        'Air Terjun Semirang', 'Obyek Wisata Goa Kreo',
        'Watu Gunung Ungaran', 'Brown Canyon',
        'Durian Gardens and Tourism Education Watu Simbar',
        'Flower Farm Setiya Aji', 'Ekowisata Mangrove Wonorejo',
        'Taman Harmoni Keputih', 'Air Mancur Menari', 'Taman Prestasi',
        'Monumen Kapal Selam', 'Taman Kunang-Kunang',
        'Taman Buah Surabaya', 'Taman Pelangi', 'Hutan Bambu Keputih',
        'Taman Keputran', 'Food Junction Grand Pakuwon',
        'Taman Barunawati', 'Kenjeran Park', 'Surabaya North Quay',
        'Kebun Bibit Wonorejo', 'Taman Ekspresi Dan Perpustakaan',
        'Kebun Binatang Surabaya', 'Rumah Batik', 'Ciputra Waterpark',
        'Masjid Muhammad Cheng Hoo', 'Masjid Nasional Al-Akbar',
        'Museum TNI AL Loka Jala Crana', 'Jembatan Merah',
        'Museum De Javasche Bank', 'Keraton Surabaya',
        'Klenteng Sanggar Agung', 'Museum Sepuluh Nopember Kota Surabaya',
        'Monumen Tugu Pahlawan', 'Surabaya Museum (Gedung Siola)',
        'Museum Kesehatan Dr. Adhyatma', 'Monumen Jalesveva Jayamahe',
        'Patung Sura dan Buaya', 'Pantai Ria Kenjeran',
        'Waterpark Kenjeran Surabaya', 'Balai Kota Surabaya',
        'Patung Buddha Empat Rupa', 'Monumen Bambu Runcing Surabaya',
        'House of Sampoerna', 'Atlantis Land Surabaya',
        'Taman Hiburan Rakyat', 'Taman Mundu', 'Museum Mpu Tantular',
        'Taman Bungkul', 'Taman Air Mancur Menari Kenjeran',
        'Taman Flora Bratang Surabaya',
        'Gereja Perawan Maria Tak Berdosa Surabaya']

In [19]:
for PlaceName in popular_tourism:
    print('Place Name:',PlaceName )
    print('Top 5 recommended PlaceName:', rekomendasi_tempat_wisata(PlaceName_title=PlaceName, PlaceName=PlaceName_list, doc_sims=doc_sim_df))
    print()

Place Name: Monumen Nasional
Top 5 recommended PlaceName: ['Museum Bahari Jakarta' 'Museum Fatahillah' 'Museum Seni Rupa dan Kramik'
 'Museum Sonobudoyo Unit I' 'Museum Wayang']

Place Name: Kota Tua
Top 5 recommended PlaceName: ['Museum Bahari Jakarta' 'Museum Fatahillah' 'Museum Seni Rupa dan Kramik'
 'Museum Sonobudoyo Unit I' 'Museum Wayang']

Place Name: Dunia Fantasi
Top 5 recommended PlaceName: ['Museum Bahari Jakarta' 'Museum Fatahillah' 'Museum Seni Rupa dan Kramik'
 'Museum Sonobudoyo Unit I' 'Museum Wayang']

Place Name: Taman Mini Indonesia Indah (TMII)
Top 5 recommended PlaceName: ['Museum Bahari Jakarta' 'Museum Fatahillah' 'Museum Seni Rupa dan Kramik'
 'Museum Sonobudoyo Unit I' 'Museum Wayang']

Place Name: Atlantis Water Adventure
Top 5 recommended PlaceName: ['Museum Bahari Jakarta' 'Museum Fatahillah' 'Museum Seni Rupa dan Kramik'
 'Museum Sonobudoyo Unit I' 'Museum Wayang']

Place Name: Taman Impian Jaya Ancol
Top 5 recommended PlaceName: ['Museum Bahari Jakarta' '

Top 5 recommended PlaceName: ['Museum Bahari Jakarta' 'Museum Fatahillah' 'Museum Seni Rupa dan Kramik'
 'Museum Sonobudoyo Unit I' 'Museum Wayang']

Place Name: Taman Begonia
Top 5 recommended PlaceName: ['Museum Bahari Jakarta' 'Museum Fatahillah' 'Museum Seni Rupa dan Kramik'
 'Museum Sonobudoyo Unit I' 'Museum Wayang']

Place Name: Tafso Barn
Top 5 recommended PlaceName: ['Museum Bahari Jakarta' 'Museum Fatahillah' 'Museum Seni Rupa dan Kramik'
 'Museum Sonobudoyo Unit I' 'Museum Wayang']

Place Name: Curug Luhur Waterfall
Top 5 recommended PlaceName: ['Museum Bahari Jakarta' 'Museum Fatahillah' 'Museum Seni Rupa dan Kramik'
 'Museum Sonobudoyo Unit I' 'Museum Wayang']

Place Name: Curug Malela
Top 5 recommended PlaceName: ['Museum Bahari Jakarta' 'Museum Fatahillah' 'Museum Seni Rupa dan Kramik'
 'Museum Sonobudoyo Unit I' 'Museum Wayang']

Place Name: Curug Cimahi
Top 5 recommended PlaceName: ['Museum Bahari Jakarta' 'Museum Fatahillah' 'Museum Seni Rupa dan Kramik'
 'Museum Sono

In [21]:
pickle.dump(rekomendasi_tempat_wisata, open("model.pkl", "wb"))